## CODE INFORMATION:

This code, given two trains, their destinations, and a time frame, will download the arrival delay data for these trains at their destinations within the time frame and run a 2-sample t-test with the trains to see if one of the given trains was significantly more delayed than the other within the time frame.

In [1]:
import requests
from matplotlib import pyplot as plt
import numpy as np
import os
import math

# Get Delays

In [2]:
# Get the stats we need from the files
def getDelays(trainNumber, endCity, start_month, start_day, start_year, end_month, end_day, end_year):
  
    delaysByTrain = { }
    
    # One of these 2 must exist, other can be ""
    # Number can be entered as "30*", "2100-2200", "3,4,5,6" or "all"
    number = str(trainNumber)
    station = endCity

    # Must all exist
    start_month = str(start_month)
    start_day = str(start_day)
    start_year = str(start_year)

    # Don't have to exist if only looking at one day
    # If only looking at one day, can be ""
    end_month = str(end_month)
    end_day = str(end_day)
    end_year = str(end_year)

    # All these must exist
    sunday = "1"
    monday = "1"
    tuesday = "1"
    wednesday = "1"
    thursday = "1"
    friday = "1"
    saturday = "1"

    # Must exist
    # Should be "d_dp" or "d_ar"
    delay_type = "d_ar"

    # Must exist
    # "DESC" or "ASC"
    order = "DESC"

    # Must exist
    # "gt", "gteq", "eq", "lteq", "lt"
    sign = "gt"

    # Optional, can be ""
    minutes = ""

    # Keep at 1
    # Setting to zero ignores weekday inputs
    # Set to zero if you always want every day of the week
    dfon = "1"

    URL = "https://juckins.net/amtrak_status/archive/html/history.php?"
    URL = URL + "train_num=" + number
    URL = URL + "&station=" + station
    URL = URL + "&date_start=" + start_month + "%2F" + start_day + "%2F" + start_year
    URL = URL + "&date_end=" + end_month + "%2F" + end_day + "%2F" + end_year
    URL = URL + "&df1=" + sunday + "&df2=" + monday + "&df3=" + tuesday + "&df4=" + wednesday + "&df5=" + thursday + "&df6=" + friday + "&df7=" + saturday
    URL = URL + "&sort=" + delay_type + "&sort_dir=" + order
    URL = URL + "&co=" + sign + "&limit_mins=" + minutes + "&dfon=" + dfon
    page = requests.get(URL)
    
    listDelays = []
    toFind = "</span></td><td style=\"text-align: center;\">"
    length = len(toFind)
    searching = page.text.find(toFind)
    while (searching != -1) :
        end = page.text.find("<", searching + length)
        delayNumber = int(page.text[searching+length:end])
        listDelays.append(delayNumber)
        searching = page.text.find(toFind, end)
        
    return listDelays

# Functions

In [15]:
# Import library
import scipy.stats as stats

In [16]:
# Use Scipy to get the p-value
def getPValue(train1Delays, train2Delays):
    train1Data = np.asarray(train1Delays)
    train2Data = np.asarray(train2Delays)
    train1Mean = train1Data.mean()
    train2Mean = train2Data.mean()

    test = stats.ttest_ind(a=train1Data, b=train2Data, equal_var=False)
    statistic = test[0]
    p_value = test[1]

    print("Test Statistic:", statistic)
    print("P-value:", p_value)
    if (train1Mean > train2Mean):
        print("If the p-value is sufficiently small,\n train #1 is significantly more delayed than train #2 on average.")
    else:
        print("If the p-value is sufficiently small,\n train #2 is significantly more delayed than train #1 on average.")

In [17]:
# Use math I have learned in ORF245 to get the p-value
def myPValue(train1Delays, train2Delays):

    train1Data = np.asarray(train1Delays)
    train2Data = np.asarray(train2Delays)
    train1Mean = train1Data.mean()
    train2Mean = train2Data.mean()


    if (len(train1Delays) >= 40 and len(train2Delays) >= 40):
        myZ = (train2Mean-train1Mean)/(np.sqrt(train1Data.var(ddof=1)/len(train1Delays) + train2Data.var(ddof=1)/len(train2Delays)))
        
        df_top = (train1Data.var(ddof=1)/len(train1Delays) + train2Data.var(ddof=1)/len(train2Delays))**2
        df_bottom_1 = 1/(len(train1Delays) - 1) * (train1Data.var(ddof=1)/len(train1Delays))**2
        df_bottom_2 = 1/(len(train2Delays) - 1) * (train2Data.var(ddof=1)/len(train2Delays))**2
        df_true = df_top / (df_bottom_1 + df_bottom_2)
        
        myP = (1 - stats.t.cdf(abs(myZ), df = df_true))*2
        
        print("My Test Statistic:", myZ)
        print("My P-value:", myP)
        
        if (train1Mean > train2Mean):
            print("If the p-value is sufficiently small,\n train #1 is significantly more delayed than train #2 on average.")
        else:
            print("If the p-value is sufficiently small,\n train #2 is significantly more delayed than train #1 on average.")
    else:
        print("One of the trains has too small a sample size!")

# Inputs - 2 Trains/Routes Must Be Independent Samples

In [14]:
trainNumber1 = 29
endCity1 = "CHI"

trainNumber2 = 30
endCity2 = "WAS"

start_month = "01"
start_day = "01"
start_year = "2021"
end_month = "12"
end_day = "31"
end_year = "2022"

train1Delays = getDelays(trainNumber1, endCity1, start_month, start_day, start_year, end_month, end_day, end_year)
train2Delays = getDelays(trainNumber2, endCity2, start_month, start_day, start_year, end_month, end_day, end_year)

# Results

In [18]:
print("Train #1: Train", trainNumber1)
print("Destination #1:", endCity1)
print("Train #1 Average Delay:", np.mean(train1Delays), "Minutes")
print("Train #2: Train", trainNumber2)
print("Destination #2:", endCity2)
print("Train #2 Average Delay:", np.mean(train2Delays), "Minutes")
print("Time Frame:", 
      start_month + "/" + start_day + "/" + start_year + " - " + end_month + "/" + end_day + "/" + end_year)
getPValue(train1Delays, train2Delays)

Train #1: Train 29
Destination #1: CHI
Train #1 Average Delay: 62.770358306188925 Minutes
Train #2: Train 30
Destination #2: WAS
Train #2 Average Delay: 58.78896103896104 Minutes
Time Frame: 01/01/2021 - 12/31/2022
Test Statistic: 0.9079978635644432
P-value: 0.3640589002307515
If the p-value is sufficiently small,
 train #1 is significantly more delayed than train #2 on average.


In [19]:
print("Train #1: Train", trainNumber1)
print("Destination #1:", endCity1)
print("Train #1 Average Delay:", np.mean(train1Delays), "Minutes")
print("Train #2: Train", trainNumber2)
print("Destination #2:", endCity2)
print("Train #2 Average Delay:", np.mean(train2Delays), "Minutes")
print("Time Frame:", 
      start_month + "/" + start_day + "/" + start_year + " - " + end_month + "/" + end_day + "/" + end_year)
myPValue(train1Delays, train2Delays)

Train #1: Train 29
Destination #1: CHI
Train #1 Average Delay: 62.770358306188925 Minutes
Train #2: Train 30
Destination #2: WAS
Train #2 Average Delay: 58.78896103896104 Minutes
Time Frame: 01/01/2021 - 12/31/2022
My Test Statistic: -0.9079978635644432
My P-value: 0.3640589002307515
If the p-value is sufficiently small,
 train #1 is significantly more delayed than train #2 on average.
